In [1]:
import json
import gzip
import re

# 20. JSONデータの読み込み

In [2]:
F_PATH="./jawiki-country.json.gz"

In [3]:
def get_data_from_json(path:str,country_name:str)->str:
    '''
    path:読み込むファイルのpath
    country_name:情報を抽出する国の名前
    '''
    with gzip.open(path,"r") as file:
        for line in file:
            data=json.loads(line)
            if data['title']==country_name:
                Text = data['text']
    return Text

In [4]:
#print(get_data_from_json(F_PATH,"イギリス"))

# 21 カテゴリ名を含む行を抽出

()->抜き出すところ  
^->行の先頭  
[[category: hoge]]って形になってる  

In [5]:
pattern=re.compile(
r'''
(^\[\[Category.*\]\].*)$
''',re.MULTILINE+re.VERBOSE)

In [6]:
Text=get_data_from_json(F_PATH,"イギリス")
res = pattern.findall(Text)
res

['[[Category:イギリス|*]]',
 '[[Category:英連邦王国|*]]',
 '[[Category:G8加盟国]]',
 '[[Category:欧州連合加盟国]]',
 '[[Category:海洋国家]]',
 '[[Category:君主国]]',
 '[[Category:島国|くれいとふりてん]]',
 '[[Category:1801年に設立された州・地域]]']

# 22　カテゴリ名の抽出

21とは行単位だったけど、名前のみ  
()で囲む場所が変わっている.  
(.*?) キャプチャしたいとこ  
(?: ) キャプチャしたくないことろ  
| はescapeしないと、AorBみたいに解釈されてしまう  

In [7]:
pattern=re.compile(
    r'''
    ^\[\[Category:(.*?)(?:\|.*)?\]\].*$
    ''',re.MULTILINE+re.VERBOSE)

In [8]:
Text=get_data_from_json(F_PATH,"イギリス")
res = pattern.findall(Text)
res

['イギリス', '英連邦王国', 'G8加盟国', '欧州連合加盟国', '海洋国家', '君主国', '島国', '1801年に設立された州・地域']

# 23 セクション構造

={2,} 直前の正規表現(=の事)を2回以上出来るだけ多くマッチさせる  
\s スペース  
(.+?)　セクション名を抜き出す  
+は一回以上の繰り返し(*は０回以上の繰り返し)
\s スペース
\1 で一つ目のグループ(=^{2,])のことを指す(これは、まっちしない)

In [9]:
pattern=re.compile(
    r'''
    ^(={2,})\s*(.+?)\s*\1.*$
    ''',re.MULTILINE+re.VERBOSE)

In [10]:
res = pattern.findall(Text)
for line in res[:3]:
    print(line)

('==', '国名')
('==', '歴史')
('==', '地理')


In [11]:
for line in res:
    level = len(line[0])-1
    print("--"*(level-1)+line[1]+" "+str(level))

国名 1
歴史 1
地理 1
--気候 2
政治 1
外交と軍事 1
地方行政区分 1
--主要都市 2
科学技術 1
経済 1
--鉱業 2
--農業 2
--貿易 2
--通貨 2
--企業 2
交通 1
--道路 2
--鉄道 2
--海運 2
--航空 2
通信 1
国民 1
--言語 2
--宗教 2
--婚姻 2
--教育 2
文化 1
--食文化 2
--文学 2
--哲学 2
--音楽 2
----イギリスのポピュラー音楽 3
--映画 2
--コメディ 2
--国花 2
--世界遺産 2
--祝祭日 2
スポーツ 1
--サッカー 2
--競馬 2
--モータースポーツ 2
脚注 1
関連項目 1
外部リンク 1


# 24 ファイル参照の抽出

* 抽出したいところは、File: hoge or ファイル: hogeってなってる

(?:　マッチしても取り出さない
(.+?)取り出したいところ(ファイル名)
ファイル名の終わりは「｜]がついているから、そこが終わりになるようにファイル名を抜き出す

In [12]:
pattern = re.compile(
    r'''
    (?:File|ファイル):(.+?)\|
    ''',re.MULTILINE+re.VERBOSE)

In [13]:
res=pattern.findall(Text)

In [14]:
res

['Royal Coat of Arms of the United Kingdom.svg',
 'Battle of Waterloo 1815.PNG',
 'The British Empire.png',
 'Uk topo en.jpg',
 'BenNevis2005.jpg',
 'Elizabeth II greets NASA GSFC employees, May 8, 2007 edit.jpg',
 'Palace of Westminster, London - Feb 2007.jpg',
 'David Cameron and Barack Obama at the G20 Summit in Toronto.jpg',
 'Soldiers Trooping the Colour, 16th June 2007.jpg',
 'Scotland Parliament Holyrood.jpg',
 'London.bankofengland.arp.jpg',
 'City of London skyline from London City Hall - Oct 2008.jpg',
 'Oil platform in the North SeaPros.jpg',
 'Eurostar at St Pancras Jan 2008.jpg',
 'Heathrow T5.jpg',
 'Anglospeak.svg',
 'CHANDOS3.jpg',
 'The Fabs.JPG',
 'PalaceOfWestminsterAtNight.jpg',
 'Westminster Abbey - West Door.jpg',
 'Edinburgh Cockburn St dsc06789.jpg',
 'Canterbury Cathedral - Portal Nave Cross-spire.jpeg',
 'Kew Gardens Palm House, London - July 2009.jpg',
 '2005-06-27 - United Kingdom - England - London - Greenwich.jpg',
 'Stonehenge2007 07 30.jpg',
 'Yard2.jpg'

# 25 テンプレートの抽出

{{基礎情報   
    hoge  
 }}の形  
re.DOTALL .が改行も含むようになる。  
*?　非貪欲マッチ  

In [15]:
def get_template_content(Text):
    '''
    jsonから
    {{基礎情報
    hoge
    }}
    のhogeの部分を抜き出してくる。
    hogeほげのことろは基礎情報が | 区切りで繋がっている
    '''
    
    pattern = re.compile(
    r'''
    ^\{\{基礎情報.*?$
    (.*?)
    ^\}\}$
    ''',re.MULTILINE+re.DOTALL+re.VERBOSE)
    contents = pattern.findall(Text)
    return contents

見出し = valの形になっている。  
(?=\n\|)で、取り出す部分の終わりを定めている。  
 (?=)を使うことで、\n\|が消費されずに、残る。  
 contentsはlistの形なので[0]で中身を取りだす

In [16]:
def parse_templete_content(contents):
    '''
    基礎情報の中身が与えられるので
    | 区切りになっているものを、辞書オブジェクトとして取り出す
    '''
    pattern = re.compile(r'''
     ^\|(.+?)\s*=\s*(.+?)
    (?:(?=\n\|)| (?=\n$))
    ''', re.MULTILINE + re.VERBOSE + re.DOTALL)
    ret = pattern.findall(contents[0])
    return ret

In [17]:
def extract_template():
    '''
    contentsから情報を抜き出す
    基礎情報の部分を取り出してreturn
    '''
    Text=get_data_from_json(F_PATH,"イギリス")
    contents = get_template_content(Text)
    ret = parse_templete_content(contents)
    return ret

In [18]:
def create_template_dic(preprocess=(lambda x:x)):
    '''
    基礎情報を辞書にする
    info=[(key_naem,val)]の形になっている
    '''
    info=extract_template()
    dic={}
    for key,val in info:
        dic[key]=preprocess(val)
    return dic

In [19]:
dic=create_template_dic()

In [20]:
for key in dic.keys():
    print("{}->{}\n".format(key,dic[key]))

略名->イギリス

日本語国名->グレートブリテン及び北アイルランド連合王国

公式国名->{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>

国旗画像->Flag of the United Kingdom.svg

国章画像->[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]

国章リンク->（[[イギリスの国章|国章]]）

標語->{{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）

国歌->[[女王陛下万歳|神よ女王陛下を守り給え]]

位置画像->Location_UK_EU_Europe_001.svg

公用語->[[英語]]（事実上）

# 26 強調マークアップの除去

強調の除去をする
''がついているやつ。

In [21]:
def remove_emph(Text):
    '''
    Text中で、single quoteで挟まれているやつをsubstractする'''
    pattern = re.compile(
    r'''
    (\'{2,5})
    (.*?)
    (\1)
    ''',re.MULTILINE+re.VERBOSE)
    return pattern.sub(r'\2',Text)
    #\2でキャプチャしたグループの二個目を取り出す。そうしないと、全部消されちゃう

In [22]:
def preprocess(val):
    '''
    強調マークあっぷをのぞく
    '''
    return remove_emph(val)

In [23]:
dic=create_template_dic(preprocess)

In [24]:
for key in dic.keys():
    print("({}-> {})\n".format(key,dic[key]))
    

(略名-> イギリス)

(日本語国名-> グレートブリテン及び北アイルランド連合王国)

(公式国名-> {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>)

(国旗画像-> Flag of the United Kingdom.svg)

(国章画像-> [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]])

(国章リンク-> （[[イギリスの国章|国章]]）)

(標語-> {{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）)

(国歌-> [[女王陛下万歳|神よ女王陛下を守り給え]])

(位置画像-> Location_UK_EU_Europe

# 27 内部リンクの削除

内部リンクは、
* [[ ___記事名___ ]]、
* [[記事名 | ___表示文字___ ]] 
* [[記事名 # | ___表示文字___ ]]
の形

In [25]:
def remove_link(Text):
    pattern = re.compile(r'''
    \[\[
    (?:
    [^|]*?\|
    )        # キャプチャしないやつ( 記事名(#)|)のとこ
    *? # ?のnon-greedy match
    ([^|]*?) # キャプチャすることろ　non-greedy
    \]\]
    ''',re.MULTILINE+re.VERBOSE)
    ret = pattern.sub(r'\1',Text)
    
    return ret

In [26]:
def preprocess(val):
    '''
    強調マークアップと内部リンク
    '''
    return remove_link(remove_emph(val))

In [27]:
dic=create_template_dic(preprocess)

In [28]:
for key in dic.keys():
    print("({}-> {})\n".format(key,dic[key]))
    

(略名-> イギリス)

(日本語国名-> グレートブリテン及び北アイルランド連合王国)

(公式国名-> {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>)

(国旗画像-> Flag of the United Kingdom.svg)

(国章画像-> イギリスの国章)

(国章リンク-> （国章）)

(標語-> {{lang|fr|Dieu et mon droit}}<br/>（フランス語:神と私の権利）)

(国歌-> 神よ女王陛下を守り給え)

(位置画像-> Location_UK_EU_Europe_001.svg)

(公用語-> 英語（事実上）)

(首都-> ロンドン)

(最大都市-> ロンドン)

(元首等肩書-> 女王)

(元首等氏名-> エリザベス2世)

(首相等肩書-> 首相)

(首相

# 28 MediaWiki マークアップの除去

In [29]:
def remove_itemize(Text):
    '''
    *とかの箇条書きを取り除く
    '''
    pattern = re.compile(r'''
    ^
    (?:[\*]{1,2} | [\#]{1,2})
    (.*)
    $
    ''',re.MULTILINE+re.VERBOSE)
    ret = pattern.sub(r'\1',Text)
    return ret

In [30]:
def remove_lang(Text):
    '''
    {{lang|hoge|info}}のやつ
    infoの部分を取り出したい 
    '''
    
    pattern = re.compile(r'''
    \{\{lang
    \|[^|]*?
    \|([^|]*?)\}\}(.*)
    ''',re.MULTILINE+re.VERBOSE)
    ret = pattern.sub(r'\1\2',Text)
    return ret

In [31]:
def remove_br_ref(Text):
    '''
    <ref>と<\ref>
    <br>と<\br>
    '''
    pattern = re.compile(
    r'''
    <
    \/? #閉じるやつ(escape /を0 or 1)
    [br|ref]
    [^>]*? #^で〜以外。>以外を非貪欲で(>が次に来るまで) <>に囲まれた中
    >
    ''',re.MULTILINE+re.VERBOSE)
    ret = pattern.sub('',Text)
    #取り出すところがないから、''
    return ret

In [32]:
def remove_exLink(Text):
    """
    http:// のやつを消す
    """
    patter = re.compile(r'''
    \[http:\/\/
    (?:
    [^\s]*?
    \s
    )?
    ([^]]*?)
    \]
    ''',re.MULTILINE+re.VERBOSE)
    ret = pattern.sub(r'',Text)
    return ret

In [33]:
def preprocess(val):
    val = remove_itemize(val)
    val = remove_link(val)
    val = remove_br_ref(val)
    val = remove_emph(val)
    val = remove_lang(val)
    val = remove_exLink(val)
    return val

In [34]:
dic = create_template_dic(preprocess)

In [35]:
for key in dic.keys():
    print("{}->{} \n".format(key,dic[key]))

略名->イギリス 

日本語国名->グレートブリテン及び北アイルランド連合王国 

公式国名->United Kingdom of Great Britain and Northern Ireland英語以外での正式国名:
An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath（スコットランド・ゲール語）
Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon（ウェールズ語）
Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann（アイルランド語）
An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh（コーンウォール語）
Unitit Kinrick o Great Breetain an Northren Ireland（スコットランド語）
Claught Kängrick o Docht Brätain an Norlin Airlann、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語） 

国旗画像->Flag of the United Kingdom.svg 

国章画像->イギリスの国章 

国章リンク->（国章） 

標語->Dieu et mon droit（フランス語:神と私の権利） 

国歌->神よ女王陛下を守り給え 

位置画像->Location_UK_EU_Europe_001.svg 

公用語->英語（事実上） 

首都->ロンドン 

最大都市->ロンドン 

元首等肩書->女王 

元首等氏名->エリザベス2世 

首相等肩書->首相 

首相等氏名->デーヴィッド・キャメロン 

面積順位->76 

面積大きさ->1 E11 

面積値->244,820 

水面積率->1.3% 

人口統計年->2011 

人口順位->22 

人口大きさ->1 E7 

人口値->63,181,775[http://esa.un.org/unpd/wpp/Excel-Data/population.htm 

# 29 国旗画像の取得

In [36]:
import urllib.parse,urllib.request

In [37]:
fname = dic['国旗画像']

In [38]:
fname

'Flag of the United Kingdom.svg'

* prop 記事の構成要素を取得
* titles 記事の名前
* file のページのurlを取得

In [39]:
req_url = 'https://www.mediawiki.org/w/api.php?' \
        + 'action=query' \
        + '&titles=File:' + urllib.parse.quote(fname) \
        + '&format=json' \
        + '&prop=imageinfo' \
        + '&iiprop=url'

In [40]:
request = urllib.request.Request(req_url)#req objを作る
connection = urllib.request.urlopen(request)# 開く


In [41]:
img_json  = json.loads(connection.read().decode())

In [42]:
url = img_json['query']['pages'].popitem()[1]['imageinfo'][0]['url']
print(url)

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
